# Oxylabs Search
## Overview

This notebook goes over how to use the Oxylabs search component.







### Integration details
| Class                                                                                                                                                    | Package |
|:---------------------------------------------------------------------------------------------------------------------------------------------------------| :--- |
| [OxylabsSearchRun](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.oxylabs_search.tool.OxylabsSearchRun.html)       | [langchain-community](https://python.langchain.com/api_reference/community/index.html) |
| [OxylabsSearchResult](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.oxylabs_search.tool.OxylabsSearchResult.html) | [langchain-community](https://python.langchain.com/api_reference/community/index.html) |


### Tool features
| [Returns artifact](/docs/how_to/tool_artifacts/) | Pricing         |
| :---: | :-----------------------:|
| ✅ | x free searches / month |

### Credentials
First, you need to set up the proper API keys and environment variables. Create your API user credentials: Sign up for a free trial or purchase the product in the [Oxylabs dashboard](https://dashboard.oxylabs.io/en/registration) to create your API user credentials (OXYLABS_USERNAME and OXYLABS_PASSWORD).



In [8]:
import getpass
import os

os.environ["OXYLABS_USERNAME"] = getpass.getpass()
os.environ["OXYLABS_PASSWORD"] = getpass.getpass()

## Instantiation
The integration lives in the `langchain-community` package. We also need to install the `oxylabs` package.

In [ ]:
%pip install -U langchain-community oxylabs

#### Basic `OxylabsSearchAPIWrapper` usage

In [9]:
from langchain_community.utilities import OxylabsSearchAPIWrapper
from pprint import pprint

api_wrapper = OxylabsSearchAPIWrapper()

In [4]:
pprint(api_wrapper.run("Python Programming Language"))

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: Redacted base64 image string...\n'
 '    IMAGES-ITEM-2: Redacted base64 image string...\n'
 '    IMAGES-ITEM-3: Redacted base64 image string...\n'
 '    IMAGES-ITEM-4: Redacted base64 image string...\n'
 '    IMAGES-ITEM-5: Redacted base64 image string...\n'
 '  FACTOIDS ITEMS: \n'
 '    FACTOIDS-ITEM-1: \n'
 '    LINKS ITEMS: \n'
 '      LINKS-ITEM-1: \n'
 '      HREF: '
 '/search?num=5&sca_esv=a9b95fc3d4d04a29&gl=us&hl=en&q=Guido+van+Rossum&stick=H4sIAAAAAAAAAONgVuLUz9U3MMwwME1exCrgXpqZkq9QlpinEJRfXFyaCwDpnKYPIAAAAA&sa=X&ved=2ahUKEwiUmPeSx8CJAxViS2wGHcqEAZoQmxMoAHoECDEQAg\n'
 '      TITLE: Guido van Rossum\n'
 '    TITLE: Designed by\n'
 '    CONTENT: Guido van Rossum\n'
 '    FACTOIDS-ITEM-2: \n'
 '    TITLE: Developer\n'
 '    CONTENT: Python Software Foundation\n'
 '    FACTOIDS-ITEM-3: \n'
 '    TITLE: Filename extensions\n'
 '    CONTENT: .py,.pyw,.pyz,.pyi,.pyc,.pyd\n'
 '    FACTOIDS-ITEM-4

#### Number of results in `OxylabsSearchAPIWrapper`
You may use the parameters `start_page`, `pages` and `limit` to specify the number of results. The default values are:  
    * `start_page = 1`  
    * `pages = 1`  
    * `limit = 5`  

In [5]:
api_wrapper = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 2,
        "limit": 10
    }
)

In [6]:
pprint(api_wrapper.run("Python Programming Language"))

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: Redacted base64 image string...\n'
 '    IMAGES-ITEM-2: Redacted base64 image string...\n'
 '    IMAGES-ITEM-3: Redacted base64 image string...\n'
 '  FACTOIDS ITEMS: \n'
 '    FACTOIDS-ITEM-1: \n'
 '    LINKS ITEMS: \n'
 '      LINKS-ITEM-1: \n'
 '      HREF: '
 '/search?sca_esv=a9b95fc3d4d04a29&gl=us&hl=en&q=Guido+van+Rossum&stick=H4sIAAAAAAAAAONgVuLUz9U3MMwwME1exCrgXpqZkq9QlpinEJRfXFyaCwDpnKYPIAAAAA&sa=X&ved=2ahUKEwig3Kahx8CJAxVzUkEAHf7XD3MQmxMoAHoECDoQAg\n'
 '      TITLE: Guido van Rossum\n'
 '    TITLE: Designed by\n'
 '    CONTENT: Guido van Rossum\n'
 '    FACTOIDS-ITEM-2: \n'
 '    TITLE: Developer\n'
 '    CONTENT: Python Software Foundation\n'
 '    FACTOIDS-ITEM-3: \n'
 '    TITLE: Filename extensions\n'
 '    CONTENT: .py,.pyw,.pyz,.pyi,.pyc,.pyd\n'
 '    FACTOIDS-ITEM-4: \n'
 '    TITLE: First appeared\n'
 '    CONTENT: 20 February 1991; 33 years ago\n'
 '    FACTOIDS-ITEM-5: \n'
 '   

#### Image Content Display in `OxylabsSearchAPIWrapper`
You may use the `include_binary_image_data` parameter to manage the display of binary image data. By default, this parameter is set to `False`, and the image data is replaced with the text "Redacted base64 image string...". 

In [7]:
api_wrapper = OxylabsSearchAPIWrapper(
    include_binary_image_data=True,
    params={
        "start_page": 1,
        "pages": 2,
        "limit": 10
    }
)
pprint(api_wrapper.run("Python Programming Language"))

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: '
 '/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIALoAugMBEQACEQEDEQH/xAAbAAEBAAIDAQAAAAAAAAAAAAAAAQUHAgQGA//EAEcQAAIBAgIECAgLBgcBAAAAAAABAgMEBREGEiExBxNBUVNhcZEWIlKBkqHB0RQVMjZVcnOUsbLSMzVCgpOzIzRDVGLw8Rf/xAAbAQEBAAMBAQEAAAAAAAAAAAAAAQMEBQYCB//EADERAQACAQIEAwYGAwEBAAAAAAABAgMEERIhMVEFExQyM0FScYEVImGRobHB0eE0Qv/aAAwDAQACEQMRAD8A3iAAAAAADjKSSzzWwDymN6c4bh8pUbRSva62NU2lBPrl7kzax6S9+c8mrk1dK8o5vJ3unuNXDaoujbRfJThm152bVdHjjrzattXknpyYW4x7F7j9rid4/q1nH1RyM8YscdKwwzlyT1tLp1Lm4qbalxXn9eo2fUREdIfPFb4vk9+be3tLKOcLivTedOvVh9Wo0TaJ6rEzHR26GOYtbv8AwcTvI9taUl3PMxzjpPWIfUZLx0mf3Zix09xy11VVqUrmK38bBJvzrIxW0uOenJnrqskfq9Zg3CDht7NUr6nOxqvdKT1qb/m5POjVvpb15xzbOPVUtytyevpyjOKlBpxe1NPYzVbTmAAAAAAAAAAAAACPcBrXhA0kq1rqeE2dSUKNLZcSg9tSXk58yOjpMEcPHZztVmmbcFX

#### Result Filtering / Ordering in `OxylabsSearchAPIWrapper`

Default result group order :  
* `knowledge_graph`  
* `combined_search_result`  
* `product_information`  
* `local_information`  
* `search_information`  
  
You may want to adjust the display order of result groups and exclude the `product_information`, `local_information`  and `combined_search_result` groups in cases where only abstract knowledge is needed. For example, this configuration allows tailored result presentation.

In [8]:
api_wrapper = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 1,
        "limit": 10,
        "geo_location": "Kairo,Egypt",
        "result_categories": [
            "search_information",
            "knowledge_graph",            
        ]
    }
)
pprint(api_wrapper.run("Visiting Great Pyramid of Giza in Egypt"), indent=4)


('  SEARCH INFORMATION: \n'
 '  QUERY: Visiting Great Pyramid of Giza in Egypt\n'
 '  GEO_LOCATION: Kairo, Ägypten\n'
 '  SHOWING_RESULTS_FOR: Visiting Great Pyramid of Giza in Egypt\n'
 '  TOTAL_RESULTS_COUNT: 6940000\n'
 '\n'
 '  RELATED SEARCHES: \n'
 '  RELATED_SEARCHES ITEMS: \n'
 '    RELATED_SEARCHES-ITEM-1: Giza Pyramids official website\n'
 '    RELATED_SEARCHES-ITEM-2: Pyramids of Giza\n'
 '    RELATED_SEARCHES-ITEM-3: Giza Pyramids ticket office\n'
 '    RELATED_SEARCHES-ITEM-4: Giza Pyramids entrance fee\n'
 '    RELATED_SEARCHES-ITEM-5: Festival Pyramids Egypt\n'
 '    RELATED_SEARCHES-ITEM-6: Pyramids of Giza facts\n'
 '    RELATED_SEARCHES-ITEM-7: Tripadvisor Pyramids of Giza\n'
 '    RELATED_SEARCHES-ITEM-8: Visit Egypt\n'
 '\n'
 '  RELATED QUESTIONS: \n'
 '  ITEMS ITEMS: \n'
 '    ITEMS-ITEM-1: \n'
 '    POS: 1\n'
 '    ANSWER: The interiors of all three pyramids of Giza are open to '
 'visitors, but each requires the purchase of a separate ticket. Although '
 'tourist

You may want to adjust the display order of result groups and exclude abstract knowledge when focusing specifically on products and local services. In the following example reorders result groups to prioritise search information and local services, omitting abstract knowledge to enhance relevance for this context.

In [10]:
api_wrapper = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 5,
        "limit": 15,
        "geo_location": "Belgium",
        "result_categories": [
            "search_information",
            "local_information",            
        ]
    }
)
pprint(api_wrapper.run("Open Working Space in Belgium."))

('  SEARCH INFORMATION: \n'
 '  QUERY: Open Working Space in Belgium.\n'
 '  GEO_LOCATION: 1348, Ottignies-Louvain-la-Neuve\n'
 '  SHOWING_RESULTS_FOR: Open Working Space in Belgium.\n'
 '  TOTAL_RESULTS_COUNT: 164000000\n'
 '  NO_RESULTS_FOR_ORIGINAL_QUERY_FOUND: True\n'
 '\n'
 '  RELATED SEARCHES: \n'
 '  RELATED_SEARCHES ITEMS: \n'
 '    RELATED_SEARCHES-ITEM-1: Free coworking space Brussels\n'
 '    RELATED_SEARCHES-ITEM-2: Spaces Tour & Taxis\n'
 '    RELATED_SEARCHES-ITEM-3: Coworking space Vilvoorde\n'
 '    RELATED_SEARCHES-ITEM-4: Firma II\n'
 '    RELATED_SEARCHES-ITEM-5: Silversquare\n'
 '    RELATED_SEARCHES-ITEM-6: Palazzo brussel\n'
 '    RELATED_SEARCHES-ITEM-7: Spaces eu\n'
 '\n'
 '  RELATED QUESTIONS: \n'
 '  ITEMS ITEMS: \n'
 '    ITEMS-ITEM-1: \n'
 '    POS: 1\n'
 '    ANSWER: Roam is an immersive platform that gives remote companies their '
 'own virtual office for their colleagues, guests, customers, and professional '
 'network.\n'
 '    SOURCE: \n'
 '      SOURCE

#### Raw Results in `OxylabsSearchAPIWrapper`

The wrapper method `results()` returns all raw results in JSON format.

In [11]:
api_wrapper = OxylabsSearchAPIWrapper()
api_wrapper.results("Python Programming Language")

[{'paid': [],
  'organic': [{'pos': 1,
    'url': 'https://www.python.org/',
    'desc': 'The official home of the Python Programming Language.',
    'title': 'Welcome to Python.org',
    'sitelinks': {'expanded': [{'url': 'https://www.python.org/about/gettingstarted/',
       'title': 'Python For Beginners'},
      {'url': 'https://docs.python.org/3/tutorial/',
       'title': 'The Python Tutorial'},
      {'url': 'https://www.python.org/downloads/', 'title': 'Downloads'},
      {'url': 'https://www.python.org/about/', 'title': 'About'},
      {'url': 'https://www.python.org/downloads/windows/',
       'title': 'Python Releases for Windows'}]},
    'url_shown': 'https://www.python.org',
    'pos_overall': 1,
    'favicon_text': 'Python.org'},
   {'pos': 2,
    'url': 'https://en.wikipedia.org/wiki/Python_(programming_language)',
    'desc': 'Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant inden

#### Tool instantiation

In [11]:
from langchain_community.tools.oxylabs_search import OxylabsSearchResults, OxylabsSearchRun
from langchain_community.utilities import OxylabsSearchAPIWrapper

api_wrapper = OxylabsSearchAPIWrapper()

#### Tool `OxylabsSearchRun`

In [13]:
tool_run = OxylabsSearchRun(wrapper=api_wrapper)

#### Tool `OxylabsSearchResult`

In [14]:
tool_results = OxylabsSearchResults(wrapper=api_wrapper)

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)
The `OxylabsSearchRun` tool takes a single "query" argument, which should be a natural language query and returns combined string format result:

In [15]:
tool_run.invoke({
    "query": "Visit restaurants in Vilnius."
})

"  ORGANIC RESULTS ITEMS: \n    ORGANIC-ITEM-1: \n    POS: 1\n    URL: https://www.tripadvisor.com/Restaurants-g274951-Vilnius_Vilnius_County.html\n    DESC: Some of the best restaurants in Vilnius for families with children include: Momo grill Vilnius · Jurgis ir Drakonas Ogmios · RoseHip Vegan\xa0...\n    TITLE: THE 10 BEST Restaurants in Vilnius (Updated November ...\n    SITELINKS: \n      SITELINKS: \n      INLINE ITEMS: \n        INLINE-ITEM-1: \n        URL: https://www.tripadvisor.com/Restaurants-g274951-zfp58-Vilnius_Vilnius_County.html\n        TITLE: Vilnius Dinner Restaurants\n        INLINE-ITEM-2: \n        URL: https://www.tripadvisor.com/Restaurants-g274951-zfp2-Vilnius_Vilnius_County.html\n        TITLE: Vilnius Breakfast Restaurants\n        INLINE-ITEM-3: \n        URL: https://www.tripadvisor.com/Restaurants-g274951-c8-Vilnius_Vilnius_County.html\n        TITLE: Cafés in Vilnius\n    URL_SHOWN: https://www.tripadvisor.com› ... › Vilnius\n    FAVICON_TEXT: Tripadviso

The `OxylabsSearchResults` tool takes a single "query" argument, which should be a natural language query and returns all raw results in JSON format:

In [16]:
import json
response_results = tool_results.invoke({
    "query": "Visit restaurants in Vilnius."
})
response_results = json.loads(response_results)
for item in response_results:
    print(item)

{'paid': [], 'organic': [{'pos': 1, 'url': 'https://www.tripadvisor.com/Restaurants-g274951-Vilnius_Vilnius_County.html', 'desc': 'Restaurants in Vilnius ; Pirmas Blynas · 316. Cafe, International, European, Healthy ; Pachamama Dinner Club · 529. Peruvian, Latin, Bar,\xa0...', 'title': 'THE 10 BEST Restaurants in Vilnius (Updated November ...', 'sitelinks': {'inline': [{'url': 'https://www.tripadvisor.com/Restaurants-g274951-zfp58-Vilnius_Vilnius_County.html', 'title': 'Vilnius Dinner Restaurants'}, {'url': 'https://www.tripadvisor.com/Restaurants-g274951-zfp2-Vilnius_Vilnius_County.html', 'title': 'Vilnius Breakfast Restaurants'}, {'url': 'https://www.tripadvisor.com/Restaurants-g274951-c8-Vilnius_Vilnius_County.html', 'title': 'Cafés in Vilnius'}]}, 'url_shown': 'https://www.tripadvisor.com› ... › Vilnius', 'pos_overall': 1, 'favicon_text': 'Tripadvisor'}, {'pos': 2, 'url': 'https://www.amsterdamfoodie.nl/2022/foodie-guide-to-vilnius-lithuania/', 'desc': 'Dec 2, 2022 — Fine dining in

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

In [12]:
tool_run = OxylabsSearchRun(
    wrapper=api_wrapper,
    kwargs={
        "result_categories": [
            "local_information",  
            "combined_search_result",
        ]
    }
)

In [13]:
model_generated_tool_call = {
        "args": {
        "query": "Visit restaurants in Vilnius.",
        "geo_location": "Vilnius,Lithuania",    
    },
    "id": "1",
    "name": "oxylabs_search",
    "type": "tool_call",
}
tool_msg = tool_run.invoke(model_generated_tool_call)

# The content is a JSON string of results
pprint(tool_msg.content[:400])

('  LOCAL PACK: \n'
 '  ITEMS ITEMS: \n'
 '    ITEMS-ITEM-1: \n'
 '    CID: 13950149882539119249\n'
 '    POS: 1\n'
 '    TITLE: Etno Dvaras\n'
 '    RATING: 4.5\n'
 '    ADDRESS: Lithuanian\n'
 '    ITEMS-ITEM-2: \n'
 '    CID: 711702509070991018\n'
 '    POS: 2\n'
 '    TITLE: Lokys\n'
 '    RATING: 4.5\n'
 '    ADDRESS: Lithuanian\n'
 '    ITEMS-ITEM-3: \n'
 '    CID: 14413965230854872799\n'
 '    POS: 3\n'
 '    TITLE: Grey\n'
 '    RATING: 4.5\n'
 '    ADDRESS: Restaurant\n'
 '\n'
 '  ORGANIC RESU')


## Chaining

In [19]:
# Install the required libraries
%pip install --upgrade --quiet langchain langchain-openai langchainhub langchain-community


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Import necessary modules
import getpass
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI  # Use OpenAIChat instead of AzureChatOpenAI

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Define assistant instructions and pull a base prompt template
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Initialize OpenAI chat model
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Define tools and agent setup
tool = OxylabsSearchRun(wrapper=api_wrapper)
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)

# Set up and invoke the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
agent_executor.invoke({"input": "What happened in the latest Burning Man floods?"})

/home/ausra/Data/AMV-workspace/projects_review/Oxylabs/langchain-project/.venv/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': 'latest Burning Man floods news'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331
    DESC: Sep 4, 2023 — Tens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend.
    TITLE: Burning Man flooding: What happened to stranded ...
    URL_SHOWN: https://abcnews.go.com› story
    FAVICON_TEXT: ABC News - Breaking News, Latest News and Videos
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://www.outsideonline.com/outdoor-adventure/exploration-survival/burning-man-2023-flooding-rain/
    DESC: Sep 6, 2023 — When storms left Burning Man 2023 a muddy mess, attendees plotted their way out.
    TITLE: How One Man Escaped From Burning Man 2023's Floods
    URL_SHOWN: https://www.outsideonline.com› exploration-survival
    FAVICON_TEXT: Outside 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The latest news regarding the Burning Man floods indicates that tens of thousands of Burning Man attendees were stranded at the festival due to a downpour and massive flooding over the weekend. Attendees were finally able to leave the festival after being stranded. Monsoon rains caused flash flooding at Burning Man, transforming the usually dry desert landscape of the festival into a muddy mess. One death was reported at the site, and authorities are investigating the incident.

For more detailed information, you can visit the following sources:
1. [ABC News - Burning Man flooding: What happened to stranded festivalgoers](https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331)
2. [Outside Magazine - How One Man Escaped From Burning Man 2023's Floods](https://www.outsideonline.com/outdoor-adventure/exploration-survival/burning-man-2023-flooding-rain/)
3. [FOX Weather - Travel to Burning Man delayed due to thunderstorms](https://www.foxweather.c

{'input': 'What happened in the latest Burning Man floods?',
 'output': "The latest news regarding the Burning Man floods indicates that tens of thousands of Burning Man attendees were stranded at the festival due to a downpour and massive flooding over the weekend. Attendees were finally able to leave the festival after being stranded. Monsoon rains caused flash flooding at Burning Man, transforming the usually dry desert landscape of the festival into a muddy mess. One death was reported at the site, and authorities are investigating the incident.\n\nFor more detailed information, you can visit the following sources:\n1. [ABC News - Burning Man flooding: What happened to stranded festivalgoers](https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331)\n2. [Outside Magazine - How One Man Escaped From Burning Man 2023's Floods](https://www.outsideonline.com/outdoor-adventure/exploration-survival/burning-man-2023-flooding-rain/)\n3. [FOX Weather 

In [21]:
agent_executor.invoke({"input": "Who won latest 2024 Lithuanian elections and why, please make an politological analysis essay from search results."})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': '2024 Lithuanian election winner', 'geo_location': 'Lithuania'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election
    DESC: Presidential elections were held in Lithuania on 12 May 2024, alongside a referendum on allowing multiple citizenships. Incumbent President Gitanas Nausėda ...
    TITLE: 2024 Lithuanian presidential election
    SITELINKS: 
      SITELINKS: 
      INLINE ITEMS: 
        INLINE-ITEM-1: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Electoral_system
        TITLE: Electoral system
        INLINE-ITEM-2: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Candidates
        TITLE: Candidates
        INLINE-ITEM-3: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Campaign
        TITLE: Campaign
        INLINE-ITEM-4: 
        URL: https://e

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': '2024 Lithuanian elections analysis', 'geo_location': 'Lithuania'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://www.robert-schuman.eu/en/monitor/6309-the-social-democrat-opposition-could-win-the-lithuanian-general-election
    DESC: Sep 17, 2024 — The Social Democrat opposition could win the Lithuanian general election ... Lithuanians will renew the 141 members of the Seimas, the single ...
    TITLE: General Elections 2024 Lithuania
    URL_SHOWN: https://www.robert-schuman.eu› monitor › 6309-the-so...
    FAVICON_TEXT: Fondation Robert Schuman
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://www.reuters.com/world/europe/lithuania-opposition-seeks-cement-election-win-run-off-votes-2024-10-27/
    DESC: 6 days ago — The opposition Social Democrats claimed victory in Lithuania's parliamentary election on Sunday, which was dominated by frustration with the ...
    TITLE: Lithuania leftist opposition win election

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### Analysis of the 2024 Lithuanian Elections

#### Election Results:
The recent Lithuanian elections in 2024 saw a significant shift in power with the opposition Social Democrats claiming victory in both the presidential and parliamentary elections. The Social Democrats won 52 seats in the 141-seat parliament, marking the end of the four-year rule of the conservative Homeland Union government. This surprise victory has reshaped the political landscape in Lithuania, leading to discussions of potential coalition formations.

#### Political Implications:
The victory of the Social Democrats signifies a notable change in public sentiment and dissatisfaction with the previous government. The election results reflect a shift towards center-left ideologies and policies, indicating a desire for change and new leadership in Lithuania. The defeat of the ruling coalition by the opposition demonstrates the electorate's demand for different governance priorities and approaches.

#### Socio-Economic

{'input': 'Who won latest 2024 Lithuanian elections and why, please make an politological analysis essay from search results.',
 'output': "### Analysis of the 2024 Lithuanian Elections\n\n#### Election Results:\nThe recent Lithuanian elections in 2024 saw a significant shift in power with the opposition Social Democrats claiming victory in both the presidential and parliamentary elections. The Social Democrats won 52 seats in the 141-seat parliament, marking the end of the four-year rule of the conservative Homeland Union government. This surprise victory has reshaped the political landscape in Lithuania, leading to discussions of potential coalition formations.\n\n#### Political Implications:\nThe victory of the Social Democrats signifies a notable change in public sentiment and dissatisfaction with the previous government. The election results reflect a shift towards center-left ideologies and policies, indicating a desire for change and new leadership in Lithuania. The defeat of th

In [22]:
agent_executor.invoke({"input": "What is the most profitable company in lithuania in 2024?"})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': 'most profitable company in Lithuania 2024', 'geo_location': 'Lithuania'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://ceoworld.biz/2024/02/19/lithuanias-largest-companies-by-market-capitalization-2024/
    DESC: Feb 19, 2024 — As of October 24, 2024, AB IGNITIS GRUPE is the most valuable company in Lithuania, with a market cap of $1.51 billion. Following it are TELIA ...
    TITLE: Lithuania's Largest Companies by market capitalization, ...
    URL_SHOWN: https://ceoworld.biz› Latest › Banking and Finance
    FAVICON_TEXT: CEOWORLD magazine
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://www.hithorizons.com/eu/analyses/country-statistics/lithuania
    DESC: Lithuania: TOP 10 Companies with Highest Sales 194,288 registered companies ; 1. ORLEN LIETUVA AB € 4.264B ; 2. MAXIMA LT UAB € 1.985B ; 3. THERMO FISHER ...
    TITLE: Industry Breakdown of Companies in Lithuania
    URL_SHOWN: https://www.hithorizons.co

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The most valuable company in Lithuania in 2024 is AB IGNITIS GRUPE with a market capitalization of $1.51 billion as of October 24, 2024.

> Finished chain.


{'input': 'What is the most profitable company in lithuania in 2024?',
 'output': 'The most valuable company in Lithuania in 2024 is AB IGNITIS GRUPE with a market capitalization of $1.51 billion as of October 24, 2024.'}

## API reference

For detailed documentation of all OxylabsSearchRun and OxylabsSearchResults features and configurations head to the API reference: 
https://python.langchain.com/api_reference/community/tools/langchain_community.tools.oxylabs_search.tool.OxylabsSearchRun.html
https://python.langchain.com/api_reference/community/tools/langchain_community.tools.oxylabs_search.tool.OxylabsSearchResult.html